# Collaborative filtering using WALS algorithm

In [0]:
PROJECT = 'qwiklabs-gcp-4a684069c4776675'
BUCKET = 'colaborative-filtering-agea'
REGION = 'us-central1'

In [0]:
import tensorflow as tf
from google.cloud import bigquery
from google.colab import auth


In [0]:
auth.authenticate_user()

In [20]:
client = bigquery.Client(project=PROJECT)
sql="""
#standardSQL
WITH visitor_page_content AS (

   SELECT  
     fullVisitorID,
     (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS latestContentId,  
     (LEAD(hits.time, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) - hits.time) AS session_duration 
   FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,   
     UNNEST(hits) AS hits
   WHERE 
     # only include hits on pages
      hits.type = "PAGE"

   GROUP BY   
     fullVisitorId, latestContentId, hits.time
     )

# aggregate web stats
SELECT   
  fullVisitorID as visitorId,
  latestContentId as contentId,
  SUM(session_duration) AS session_duration 
 
FROM visitor_page_content
  WHERE latestContentId IS NOT NULL 
  GROUP BY fullVisitorID, latestContentId
  HAVING session_duration > 0
  ORDER BY latestContentId 
"""

df = client.query(sql).to_dataframe()
df.head()

,visitorId,contentId,session_duration
0,7337153711992174438,100074831,44652
1,5190801220865459604,100170790,1214205
2,5874973374932455844,100510126,32109
3,2293633612703952721,100510126,47744
4,1173698801255170595,100676857,10512


In [23]:
sql = """
SELECT * FROM AGEA_ASL.Dataset_B LIMIT 10
"""
client.query(sql).to_dataframe().head()

,rownum,content_id,day,user_id
0,43237075,sVu74bbfv,2019-01-01,3709618
1,43237470,NaTdXsf-I,2019-01-01,3881617
2,43238282,pyb5gkGxe,2019-01-01,3200261
3,43238638,w_UpfsohQ,2019-01-01,3468920
4,43239507,PjBv31CcC,2019-01-01,5487623
